## Practice 2
---

---
#### Overview
For this practice, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.

You need to have Apache Spark installed.

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located. The code in the following cell tells Python how to find this Spark installation. This code creates SparkContext (sc) for you.

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster, and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Next, let's test that your `SparkContext` has been set up properly by running some simple test code.   This code uses a single Spark job to estimate the value of Euler's number $e$. One way to calculate $e$ is to use the following serries by Jacob Bernoulli:

$p_n = 1 - \frac{1}{1!} + \frac{1}{2!} - \frac{1}{3!} + \cdots + \frac{(-1)^n}{n!} = \sum_{k = 0}^n \frac{(-1)^k}{k!}$

As n tends to infinity, $p_n$ approaches $1/e$.

In the following code,  `parallelize()` and `map()` are Spark *transformations*, and `reduce()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take several seconds, since a Spark job is being created and executed, and should print an estimate of $e$ when it finishes.   


In [2]:
import math

n= 10000
inverse_e = sc.parallelize(range(0, n)).map(lambda x: ((-1)**x) * (1 / math.factorial(x))).reduce(lambda x,y:x+y)
e = 1 / inverse_e
print("e = ", e)

e =  2.718281828459044


---
#### Question 1:

In the following cell, briefly explain how the example works.   What is the Spark job doing, and how is it used to estimate the value of $e$? How accurate is our estimate?

#### Your answer to Question 1:

This code is used to approximate the value of Euler's number, e. 

The parallelize function distributes a local Python collection to form an RDD using the given range, [0 - 10000), and allows calculation in parallel. 

Then the map function applies a mathematical expression, which is used to approximate $e^{-1}$, on each element of the dataset.

After that, the reduce function aggregates the results by adding all the processed elements up and get a final approximation to $e^{-1}$.

Lastly inversing the result yields an approximation to Euler's number.

The general expression to calculate inverse_e is by summing all the elements in the expression from n = 0 to infinity. Here we only sum up the first 10000 elements. Note the values of the expression $\frac{1}{n!}$ becomes very small after n = 10000, which are neglectable, so in the end we still get a pretty accurate approximation to $e^{-1}$.

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

Note you should do as much as possible using RDD transformations and actions, and little-to-nothing in the driver itself. 

Example:

`someRDD.collect()[:10]` - this is bad! It's collecting *all* of the data onto the driver when we were only looking for 10  
`someRDD.take(10)` - much better!

Extra examples of what not to do:

`sc.parallelize(myFile.readlines())` - no! Have the cluster load the file using `sc.textFile()` instead!

`newRDD = sc.parallelize(<some computation>.collect())` - no!  
`newRDD = <some computation>` - yes!

---
#### Question 2:

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`. 

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Practice 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [3]:
# A2Q2 BOX
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
    lines = sc.textFile('Shakespeare.txt')
    counts = lines.flatMap(lambda line: simple_tokenize(line))\
                  .map(lambda token: (token, 1))\
                  .reduceByKey(lambda x, y: x + y)\
                  .count()
    return counts


---
#### Question 3:

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.



In [4]:
# A2Q3 BOX
from simple_tokenize import simple_tokenize

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    def token_pair(str_lst):
        str_set = set(str_lst)
        result = []
        for str1 in str_set:
            for str2 in str_set:
                if str1 == str2:
                    continue
                result.append(((str1, str2), 1))
        return result
    lines = sc.textFile('Shakespeare.txt')
    counts = lines.map(lambda line: simple_tokenize(line))\
                  .flatMap(token_pair)\
                  .reduceByKey(lambda x, y: x + y)\
                  .count()
    return counts


---
#### Question 4:

Next, write Spark code that will calculate the probability $p(x)$ for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [5]:
# A2Q4 BOX
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
    lines = sc.textFile('Shakespeare.txt')
    line_counts = lines.count()
    result = lines.flatMap(lambda line: set(simple_tokenize(line)))\
                  .map(lambda token: (token, 1))\
                  .reduceByKey(lambda x, y: x + y)\
                  .map(lambda pair: (pair[1]/line_counts, pair[1], pair[0]))\
                  .sortBy(lambda tuple: tuple[0], ascending = False)\
                  .take(50)
    return result


Hint: "and" is the most frequent token, and "the" is close.

---
#### Question 5:

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

Note: You're using a "pairs" approach here, since the key is a pair of tokens!

In [6]:
# A2Q5 BOX
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    def token_pair(str_set):
        result = []
        for str1 in str_set:
            for str2 in str_set:
                if str1 == str2:
                    continue
                result.append(((str1, str2), 1))
        return result
    def pmi(x, y, xy, line_count):
        p_x = x / line_count
        p_y = y / line_count
        p_xy = xy / line_count
        return log(p_xy/(p_x * p_y), 10)
    lines = sc.textFile('Shakespeare.txt')
    line_count = lines.count()
    tokens = lines.flatMap(lambda line: set(simple_tokenize(line)))\
                  .map(lambda token: (token, 1))\
                  .reduceByKey(lambda x, y: x + y).collect()
    tokens_bct = sc.broadcast(dict(tokens))
    token_pairs = lines.map(lambda line: set(simple_tokenize(line)))\
                       .flatMap(token_pair)\
                       .reduceByKey(lambda x, y: x + y)\
                       .filter(lambda tuple: tuple[1] >= threshold)
    result = token_pairs.map(lambda pair: (pair[0],
                                           pmi(tokens_bct.value[pair[0][0]],
                                               tokens_bct.value[pair[0][1]],
                                               pair[1],
                                               line_count),
                                           pair[1], 
                                           tokens_bct.value[pair[0][0]],
                                           tokens_bct.value[pair[0][1]]))\
                         .collect()
    return result


---
#### Question 6:

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$, with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen uniformly at random from among the $x$'s with a non-empty list of co-occurrences.

If there are fewer than $N$ different $x$'s, then the function should output all of them.  In other words, if $T$ is very large, the function would output nothing.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```

Hint: Sampling must be done at the very last step.

Hint: there is an action that returns a sample subset from an RDD. (`takeSample`)

Note: You should be taking a "stripes" approach here. While you *can* use a pairs approach and then convert to stripes at the end, it's not as efficient.

In [7]:
# A2Q6 BOX
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    def token_stripe(str_set):
        result = []
        for str1 in str_set:
            dict = {}
            for str2 in str_set:
                if str1 != str2:
                    dict[(str1, str2)] = 1
            result.append((str1, dict))
        return result
    def mergedict(d1, d2):
        keys = set(d1) | set(d2)
        result = {}
        for key in keys:
            result[key] = d1.get(key, 0) + d2.get(key, 0)
        return result
    def stripefilter(stripe):
        token = stripe[0]
        list_pairs = []
        for pair in stripe[1].items():
            if pair[1] >= threshold:
                list_pairs.append(pair)
        return (token, list_pairs)
    lines = sc.textFile('Shakespeare.txt')
    line_count = lines.count()
    tokens = lines.flatMap(lambda line: set(simple_tokenize(line)))\
                  .map(lambda token: (token, 1))\
                  .reduceByKey(lambda x, y: x + y).collect()
    tokens_bct = sc.broadcast(dict(tokens))
    
    token_stripes = lines.map(lambda line: set(simple_tokenize(line)))\
                         .flatMap(token_stripe)\
                         .reduceByKey(mergedict)
    token_stripes_threshold = token_stripes.map(stripefilter).filter(lambda pair: pair[1] != [])
    def pmi(stripe):
        list = stripe[1]
        x = 0
        y = 0
        xy = 0
        token1 = ''
        token2 = ''
        result = []
        for pair in list:
            token1 = pair[0][0]
            token2 = pair[0][1]
            xy = pair[1]
            x = tokens_bct.value[token1]
            y = tokens_bct.value[token2]
            p_x = x / line_count
            p_y = y / line_count
            p_xy = xy / line_count
            result.append(((token1, token2), log(p_xy/(p_x * p_y), 10), xy, x, y))
        return result
    result = token_stripes_threshold.map(lambda stripe: (stripe[0], pmi(stripe))).takeSample(False, samp_size)
    return result
